## Injection benchmark

CPython 3.6.5 and an i7 7700K were used for the timings

In [1]:
import antidote
import attr
%load_ext cython

antidote.global_container = antidote.new_container()
world = antidote.global_container

In [2]:
@antidote.register(tags=['test'])
class Service1:
    pass


@antidote.register(tags=['test'])
class Service2:
    def __init__(self, service1: Service1):
        self.service1 = service1
       
 
@antidote.register(tags=['test'])
class Service3:
    def __init__(self, service1: Service1, service2: Service2):
        self.service1 = service1
        self.service2 = service2

  
@antidote.register(tags=['test'])
class Service4:
    def __init__(self, service1: Service1, service2: Service2, service3: Service3):
        self.service1 = service1
        self.service2 = service2
        self.service3 = service3

@antidote.resource
def conf(key):
    return None

In [3]:
tag_provider = world.providers[antidote.TagProvider]

In [5]:
tagged = antidote.Tagged('test2')
%timeit tag_provider.provide(tagged)

949 ns ± 5.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [5]:
tagged = antidote.Tagged('test2')
%timeit tag_provider.provide(tagged)

967 ns ± 7.88 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [4]:
tagged = antidote.Tagged('test')
%timeit tag_provider.provide(tagged)

1.21 µs ± 8.14 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [4]:
tagged = antidote.Tagged('test')
%timeit tag_provider.provide(tagged)

1.24 µs ± 12.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [4]:
tagged = antidote.Tagged('test')
%timeit tag_provider.provide(tagged)

1.35 µs ± 5.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [3]:
resource_provider = world.providers[antidote.ResourceProvider]

In [4]:
%timeit resource_provider.provide("conf:test")

623 ns ± 15.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [4]:
%timeit resource_provider.provide("conf:test")

777 ns ± 6.18 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Function call

Injection overhead is here measured with a function which does nothing.

In [3]:
factory_provider = world.providers[antidote.FactoryProvider]

In [4]:
%timeit factory_provider.provide(Service1)

408 ns ± 4.54 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [6]:
%timeit factory_provider.provide(Service1)

448 ns ± 1.66 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [4]:
%timeit factory_provider.provide(Service1)

599 ns ± 7.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [3]:
def f(s1: Service1, s2: Service2, s3: Service3, s4: Service4):
    pass
f_injected = antidote.inject(f)

Time necessary to only execute the function, without retrieving the services

In [4]:
args = (world[Service1], world[Service2], world[Service3], world[Service4])
# %timeit f(*args)

Overhead of the injection when all argument must be retrieved from the container.

In [5]:
%timeit f_injected()

1.41 µs ± 25.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [6]:
%timeit f_injected()

888 ns ± 7.16 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [5]:
%timeit f_injected()

838 ns ± 7.16 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [6]:
f_injected = antidote.inject(f)
%timeit f_injected()

1.05 µs ± 12.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Overhead of the injection when no argument has to be retrieved.

In [7]:
%timeit f_injected(*args)

260 ns ± 2.75 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


If speed is critical, arguments can be bound with a `functools.partial`.

It should be noted that this is the worst scenario possible. In a real case example, the function would be much slower.
To put those results into perspective, the overhead is roughly the time needed to decode this simple JSON.

In [8]:
import json
%timeit json.loads('{ "name":"John", "age":30, "city":"New York"}')

4.5 µs ± 79.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Object instantiation

A similar benchmark is done with object instantiation.

In [9]:
class Obj:
    s1: Service1
    s2: Service2
    s3: Service3
    s4: Service4
        
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit Obj(*args)

679 ns ± 1.49 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
class ObjInjected:
    s1: Service1
    s2: Service2
    s3: Service3
    s4: Service4
        
    @antidote.inject
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit ObjInjected()

6.5 µs ± 15.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
@attr.s
class ObjAttrs:
    s1: Service1 = antidote.attrib()
    s2: Service2 = antidote.attrib()
    s3: Service3 = antidote.attrib()
    s4: Service4 = antidote.attrib()
        
%timeit ObjAttrs()

3.48 µs ± 10.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Conclusion

The key take away from those benchmarks, is to avoid using injection on short functions which are called repeatedly, in a loop typically. In the other cases, the overhead is considered to be negligeable.

Yet should it not be the case, when measured with a profiling tool, consider to either use the option `bind=True` or to retrieve the necessary services beforehand.

In [31]:
%%cython -a --cplus
# cython: language_level=3, language=c++
# cython: boundscheck=False, wraparound=False, annotation_typing=False

cimport cython
from cpython.object cimport PyObject_IsInstance


cdef class Dependency:
    """"""

@cython.freelist(10)
cdef class WrappedDependency(Dependency):
    cdef:
        readonly object wrapped

    def __init__(self, wrapped):
        self.wrapped = wrapped

    def __repr__(self):
        return "{}(wrapped={!r})".format(type(self).__name__, self.wrapped)

    def __str__(self):
        return str(self.wrapped)

    def __hash__(self):
        return hash(self.wrapped)

    def __eq__(self, object other):
        return isinstance(other, WrappedDependency) and (self.wrapped is other.wrapped 
                                                                  or self.wrapped == other.wrapped)


In [32]:
o = object()
w1 = WrappedDependency(o)
w2 = WrappedDependency(o)

In [33]:
%timeit w1 == w2

147 ns ± 0.942 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [24]:
%timeit w1 == w2

154 ns ± 0.98 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [21]:
%timeit w1 == w2

153 ns ± 1.09 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [17]:
%timeit w1 == w2

146 ns ± 1.52 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [7]:
class Dummy:
    def method(self, x):
        return self, x

    @classmethod
    def class_method(cls, x):
        return cls, x

    @staticmethod
    def static(x):
        return x

In [27]:
d = Dummy()
cm = Dummy.method
cm is cm.__get__(d, Dummy)

False

In [32]:
Dummy.__dict__['class_method'].__func__

<function __main__.Dummy.class_method(cls, x)>

In [10]:
d.method is d.method

False

In [12]:
Dummy.method is Dummy.method

True

In [33]:
b = True
id(b)

94517558804736

In [34]:
b = False
id(b)

94517558804768

In [70]:
class A:
    def __init__(self, container):
        pass
    
class B(A):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
class C:
    @classmethod
    def f(cls):
        pass
    
    @staticmethod
    def g():
        pass

In [73]:
C.__dict__['g'].__func__

<function __main__.C.g()>

In [71]:
C.g.__func__

AttributeError: 'function' object has no attribute '__func__'

In [41]:
C.__dict__.get('__call__')

In [43]:
'__call__' in A.__dict__

True

In [50]:
B.__mro__

(__main__.B, __main__.A, object)

In [53]:
A.__class__

type

In [55]:
A.__dict__['test'] = 1

TypeError: 'mappingproxy' object does not support item assignment